In [ ]:
import pandas as pd
import os

base = "/Users/authorname/Downloads/syn65414912"

# Load the key files
df_levels_anml   = pd.read_csv(os.path.join(base, "OhNM2025_ROSMAP_plasma_Soma7k_protein_level_ANML_log10.csv"))
df_levels_log10  = pd.read_csv(os.path.join(base, "OhNM2025_ROSMAP_plasma_Soma7k_protein_level_log10.csv"))
df_protein_meta  = pd.read_csv(os.path.join(base, "OhNM2025_ROSMAP_plasma_Soma7k_protein_metadata.csv"))
df_sample_meta   = pd.read_csv(os.path.join(base, "OhNM2025_ROSMAP_plasma_Soma7k_sample_metadata.csv"))

# Quick checks
print("Protein levels (ANML):", df_levels_anml.shape)
print("Protein levels (log10):", df_levels_log10.shape)
print("Protein metadata:", df_protein_meta.shape)
print("Sample metadata:", df_sample_meta.shape)

# Preview sample metadata (patients)
print("\n=== Sample Metadata Head ===")
print(df_sample_meta.head())

# Preview protein metadata
print("\n=== Protein Metadata Head ===")
print(df_protein_meta.head())

In [ ]:
# =========================
# BLOCK 1 — PREPROCESSING
# =========================
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# ---- File paths ----
BASE = "/Users/authorname/Downloads/syn65414912"
ANML_PATH   = os.path.join(BASE, "OhNM2025_ROSMAP_plasma_Soma7k_protein_level_ANML_log10.csv")
SAMPLE_PATH = os.path.join(BASE, "OhNM2025_ROSMAP_plasma_Soma7k_sample_metadata.csv")

# ---- Load ----
df_levels = pd.read_csv(ANML_PATH)        # rows: projid_visit, cols: proteins
df_meta   = pd.read_csv(SAMPLE_PATH)      # contains projid_visit, projid, msex, age_at_visit, educ, apoe_genotype, Diagnosis

# ---- Sanity: keys present? ----
assert "projid_visit" in df_levels.columns, "projid_visit missing in protein matrix."
for col in ["projid_visit","projid","msex","age_at_visit","educ","apoe_genotype","Diagnosis"]:
    assert col in df_meta.columns, f"{col} missing in sample metadata."

# ---- Align on visit ----
# inner join so we only keep visits present in BOTH
df = pd.merge(df_meta, df_levels, on="projid_visit", how="inner", validate="one_to_one")
print("Aligned shape:", df.shape)

# ---- Labels (four groups) ----
df["Diagnosis"] = df["Diagnosis"].astype(str).str.strip()
valid_classes = {"MCI","NCI","AD","AD+"}
df = df[df["Diagnosis"].isin(valid_classes)].reset_index(drop=True)
print("Class counts:\n", df["Diagnosis"].value_counts())

# ---- Grouping key for leakage control ----
df["projid"] = df["projid"].astype(str)

# ---- Stratification label: Diagnosis × sex (0/1) ----
df["msex"] = df["msex"].astype(int)
df["strata"] = df["Diagnosis"] + "_" + df["msex"].astype(str)

# ---- APOE one-hot (Unknown for NaN) ----
def format_apoe(x):
    if pd.isna(x):
        return "Unknown"
    try:
        # e.g., 33.0 -> "33"
        return str(int(float(x)))
    except Exception:
        s = str(x).strip()
        return s if s else "Unknown"

df["apoe_str"] = df["apoe_genotype"].apply(format_apoe)

# scikit-learn compatibility across versions
try:
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
except TypeError:
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")

apoe_ohe = ohe.fit_transform(df[["apoe_str"]])
apoe_cols = [c.replace("apoe_str_","APOE_") for c in ohe.get_feature_names_out()]
df_apoe  = pd.DataFrame(apoe_ohe, columns=apoe_cols, index=df.index)

# ---- Numeric covariates (unscaled) ----
for col in ["age_at_visit","educ"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ---- Protein feature columns ----
protein_cols = [c for c in df_levels.columns if c != "projid_visit"]

# ---- Final feature matrix X ----
X = pd.concat([df[["age_at_visit","educ"]], df_apoe, df[protein_cols]], axis=1)
y = df["Diagnosis"].astype(str).values
groups = df["projid"].values
strata = df["strata"].values

print("X shape:", X.shape)
print("APOE levels seen:", sorted(set(df["apoe_str"])))

In [ ]:
# === VISIT-LEVEL SUMMARY (standalone) ===
print("\n=== Visit-level summary ===")
print(f"Total patient visits: {len(df)}")

# Visits per diagnosis class (NCI, MCI, AD, AD+), missing classes shown as 0
order = ["NCI", "MCI", "AD", "AD+"]
counts = df["Diagnosis"].value_counts().reindex(order).fillna(0).astype(int)

print("Visits per class:")
for cls, cnt in counts.items():
    print(f"  {cls}: {cnt}")

In [ ]:
# =========================
# BLOCK 2 — TRAINING RF
# =========================
import os
import numpy as np
import pandas as pd
from flaml import AutoML
from sklearn.metrics import roc_auc_score

# ---- Output folder ----
out_dir = "/Users/authorname/Desktop/Projects/proteomics_RF(ANML+Meta)"
os.makedirs(out_dir, exist_ok=True)

# ---- Helper: group-aware stratified 70/30 split (by mode of strata per subject) ----
def group_stratified_shuffle_split(df_index, strata_all, groups_all, test_size=0.30, random_state=0):
    """Return train_idx, test_idx ensuring groups stay intact and class-sex balance is approx preserved.
       Strategy: assign each group a single stratum = mode(strata) among its rows, then stratified split at group level.
    """
    rng = np.random.RandomState(random_state)
    data = pd.DataFrame({"idx": df_index, "strata": strata_all, "group": groups_all})

    # group -> mode(strata)
    grp_mode = (
        data.groupby("group")["strata"]
        .agg(lambda s: s.value_counts().idxmax())
    )
    grp_mode = grp_mode.sample(frac=1.0, random_state=random_state)  # shuffle groups

    train_groups, test_groups = [], []
    for s_val, grp_ids in grp_mode.groupby(grp_mode.values):
        g_list = list(grp_ids.index)
        rng.shuffle(g_list)
        n_test = max(1, int(round(test_size * len(g_list))))
        test_groups.extend(g_list[:n_test])
        train_groups.extend(g_list[n_test:])

    train_mask = np.isin(groups_all, train_groups)
    test_mask  = np.isin(groups_all,  test_groups)
    return np.where(train_mask)[0], np.where(test_mask)[0]

# ---- Classes & seeds ----
classes = ["MCI","NCI","AD","AD+"]     # will save AD+ as 'ADplus' in filenames
def safe_cls(c): return c.replace("+","plus").replace(" ","_").replace("/","-")

seeds = [1,2,3,4,5]

# ---- Main loop ----
for seed in seeds:
    tr_idx, te_idx = group_stratified_shuffle_split(
        df_index=np.arange(len(X)),
        strata_all=strata,
        groups_all=groups,
        test_size=0.30,
        random_state=seed,
    )

    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train_full    = y[tr_idx]
    y_test_full     = y[te_idx]

    print(f"\n[Seed {seed}] Train n={len(tr_idx)} | Test n={len(te_idx)} | "
          f"Groups train={len(np.unique(groups[tr_idx]))} test={len(np.unique(groups[te_idx]))}")

    for cls in classes:
        y_train = (y_train_full == cls).astype(int)
        y_test  = (y_test_full  == cls).astype(int)

        automl = AutoML()
        settings = {
            "time_budget": 100,               # seconds per class; adjust if you want longer searches
            "metric": "roc_auc",
            "task": "classification",
            "eval_method": "cv",
            "estimator_list": ["rf"],       # force random forest
            "log_file_name": os.path.join(out_dir, f"flaml_seed{seed}_{safe_cls(cls)}.log"),
            "seed": seed,
        }
        automl.fit(X_train=X_train, y_train=y_train, **settings)
        automl.pickle(os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}_automl.pkl"))

        # Predict proba for positive class
        y_score = automl.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_score) if (y_test.sum() > 0 and y_test.sum() < len(y_test)) else float("nan")
        print(f"  [{cls}] AUC={auc:.3f}")

        # Save per-class CSV for your existing averaging/plotting script
        out_csv = os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}.csv")
        pd.DataFrame({"y_true": y_test.astype(int), "y_score": y_score.astype(float)}).to_csv(out_csv, index=False)

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# ---- Mean ROC across seeds from saved CSVs ----
folder = "/Users/authorname/Desktop/Projects/proteomics_RF(ANML+Meta)"
files = glob.glob(os.path.join(folder, "seed*_*.csv"))
assert len(files) > 0, "No per-seed ROC CSVs found."

# Infer class names from filenames (everything after 'seedX_')
def class_from_path(p):
    base = os.path.basename(p)
    return "_".join(base.split("_")[1:]).rsplit(".csv", 1)[0]

class_names = sorted(set(class_from_path(p) for p in files))

fpr_grid = np.linspace(0.0, 1.0, 200)

plt.figure(figsize=(10, 8))

for cls in class_names:
    cls_files = sorted(glob.glob(os.path.join(folder, f"seed*_{cls}.csv")))
    if not cls_files:
        continue

    tprs_interp, aucs = [], []

    for path in cls_files:
        df = pd.read_csv(path)
        y_true  = df["y_true"].astype(int).values
        y_score = df["y_score"].astype(float).values
        if np.unique(y_true).size < 2:
            # degenerate split; skip so it doesn't skew means
            continue

        # MATCH ratios convention: raw fpr/tpr, no dedup, no forcing tpr[-1]=1.0
        fpr, tpr, _ = roc_curve(y_true, y_score)

        # Interpolate to common grid for plotting only
        tpr_i = np.interp(fpr_grid, fpr, tpr)
        tpr_i[0] = 0.0
        tprs_interp.append(tpr_i)

        # Scalar AUC from raw fpr/tpr
        aucs.append(auc(fpr, tpr))

    if not tprs_interp:
        continue

    mean_tpr = np.mean(tprs_interp, axis=0)
    std_tpr  = np.std(tprs_interp, axis=0)
    mean_auc = float(np.mean(aucs))
    std_auc  = float(np.std(aucs))

    plt.plot(
        fpr_grid, mean_tpr, lw=2,
        label=f"{cls.replace('_',' ')} (AUC = {mean_auc:.3f} ± {std_auc:.3f})"
    )
    plt.fill_between(
        fpr_grid,
        np.clip(mean_tpr - std_tpr, 0, 1),
        np.clip(mean_tpr + std_tpr, 0, 1),
        alpha=0.15
    )

# Chance line
plt.plot([0, 1], [0, 1], 'k--', label='Chance (AUC = 0.50)')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('RF (Proteomics + Metadata)')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# =========================
# BLOCK 2 — TRAINING
# =========================
import os
import numpy as np
import pandas as pd
from flaml import AutoML
from sklearn.metrics import roc_auc_score

# ---- Output folder ----
out_dir = "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)"
os.makedirs(out_dir, exist_ok=True)

# ---- Helper: group-aware stratified 70/30 split (by mode of strata per subject) ----
def group_stratified_shuffle_split(df_index, strata_all, groups_all, test_size=0.30, random_state=0):
    """Return train_idx, test_idx ensuring groups stay intact and class-sex balance is approx preserved.
       Strategy: assign each group a single stratum = mode(strata) among its rows, then stratified split at group level.
    """
    rng = np.random.RandomState(random_state)
    data = pd.DataFrame({"idx": df_index, "strata": strata_all, "group": groups_all})

    # group -> mode(strata)
    grp_mode = (
        data.groupby("group")["strata"]
        .agg(lambda s: s.value_counts().idxmax())
    )
    grp_mode = grp_mode.sample(frac=1.0, random_state=random_state)  # shuffle groups

    train_groups, test_groups = [], []
    for s_val, grp_ids in grp_mode.groupby(grp_mode.values):
        g_list = list(grp_ids.index)
        rng.shuffle(g_list)
        n_test = max(1, int(round(test_size * len(g_list))))
        test_groups.extend(g_list[:n_test])
        train_groups.extend(g_list[n_test:])

    train_mask = np.isin(groups_all, train_groups)
    test_mask  = np.isin(groups_all,  test_groups)
    return np.where(train_mask)[0], np.where(test_mask)[0]

# ---- Classes & seeds ----
classes = ["MCI","NCI","AD","AD+"]     # will save AD+ as 'ADplus' in filenames
def safe_cls(c): return c.replace("+","plus").replace(" ","_").replace("/","-")

seeds = [1,2,3,4,5]

# ---- Main loop ----
for seed in seeds:
    tr_idx, te_idx = group_stratified_shuffle_split(
        df_index=np.arange(len(X)),
        strata_all=strata,
        groups_all=groups,
        test_size=0.30,
        random_state=seed,
    )

    X_train, X_test = X.iloc[tr_idx], X.iloc[te_idx]
    y_train_full    = y[tr_idx]
    y_test_full     = y[te_idx]

    print(f"\n[Seed {seed}] Train n={len(tr_idx)} | Test n={len(te_idx)} | "
          f"Groups train={len(np.unique(groups[tr_idx]))} test={len(np.unique(groups[te_idx]))}")

    for cls in classes:
        y_train = (y_train_full == cls).astype(int)
        y_test  = (y_test_full  == cls).astype(int)

        automl = AutoML()
        settings = {
            "time_budget": 100,               # seconds per class; adjust if you want longer searches
            "metric": "roc_auc",
            "task": "classification",
            "eval_method": "cv",
            "estimator_list": ["lgbm"],       # force LightGBM
            "log_file_name": os.path.join(out_dir, f"flaml_seed{seed}_{safe_cls(cls)}.log"),
            "seed": seed,
        }
        automl.fit(X_train=X_train, y_train=y_train, **settings)
        automl.pickle(os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}_automl.pkl"))

        # Predict proba for positive class
        y_score = automl.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_score) if (y_test.sum() > 0 and y_test.sum() < len(y_test)) else float("nan")
        print(f"  [{cls}] AUC={auc:.3f}")

        # Save per-class CSV for your existing averaging/plotting script
        out_csv = os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}.csv")
        pd.DataFrame({"y_true": y_test.astype(int), "y_score": y_score.astype(float)}).to_csv(out_csv, index=False)

In [ ]:
import os
import pickle
import pandas as pd
from collections import Counter

# ---- Config ----
out_dir = "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)"
classes = ["MCI", "NCI", "AD", "AD+"]
seeds = [1, 2, 3, 4, 5]

def safe_cls(c): return c.replace("+", "plus").replace(" ", "_").replace("/", "-")

# ---- Track all features with non-zero importance ----
feature_counts = Counter()

for seed in seeds:
    for cls in classes:
        model_path = os.path.join(out_dir, f"seed{seed}_{safe_cls(cls)}_automl.pkl")
        if not os.path.exists(model_path):
            print(f"Missing: {model_path}")
            continue

        with open(model_path, "rb") as f:
            automl = pickle.load(f)

        # Get feature importances
        importances = automl.model.estimator.feature_importances_
        features = automl.feature_names_in_

        # Filter to non-zero importance
        nonzero_features = [feat for feat, imp in zip(features, importances) if imp > 0]

        # Update count for each non-zero feature
        feature_counts.update(nonzero_features)

# ---- Filter to features appearing in at least 3 models ----
shared_features = [feat for feat, count in feature_counts.items() if count >= 3]
print(f"\n Found {len(shared_features)} features with non-zero importance in ≥3 models.\n")
print(shared_features)

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# ---- Update this to match your proteomics run ----
folder = "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)"
files = glob.glob(os.path.join(folder, "seed*_*.csv"))

# Infer class names from filenames (everything after 'seedX_')
def class_from_path(p):
    base = os.path.basename(p)
    return "_".join(base.split("_")[1:]).rsplit(".csv", 1)[0]

class_names = sorted(set(class_from_path(p) for p in files))

fpr_grid = np.linspace(0.0, 1.0, 200)

plt.figure(figsize=(10, 8))

for cls in class_names:
    cls_files = sorted(glob.glob(os.path.join(folder, f"seed*_{cls}.csv")))
    if not cls_files:
        continue

    tprs_interp, aucs = [], []

    for path in cls_files:
        df = pd.read_csv(path)
        y_true  = df["y_true"].astype(int).values
        y_score = df["y_score"].astype(float).values

        # EXACTLY like your ratios plot: use raw fpr/tpr directly
        fpr, tpr, _ = roc_curve(y_true, y_score)

        # Interpolate to common grid (no FPR de-dup)
        tpr_i = np.interp(fpr_grid, fpr, tpr)
        tpr_i[0] = 0.0  # match your ratios script
        tprs_interp.append(tpr_i)

        # AUC computed on raw fpr/tpr (no de-dup)
        aucs.append(auc(fpr, tpr))

    if not tprs_interp:
        continue

    mean_tpr = np.mean(tprs_interp, axis=0)
    std_tpr  = np.std(tprs_interp, axis=0)
    mean_auc = float(np.mean(aucs))
    std_auc  = float(np.std(aucs))

    plt.plot(
        fpr_grid, mean_tpr, lw=2,
        label=f"{cls.replace('_',' ')} (AUC = {mean_auc:.3f} ± {std_auc:.3f})"
    )
    plt.fill_between(
        fpr_grid,
        np.clip(mean_tpr - std_tpr, 0, 1),
        np.clip(mean_tpr + std_tpr, 0, 1),
        alpha=0.15
    )

# Chance line
plt.plot([0, 1], [0, 1], 'k--', label='Chance (AUC = 0.50)')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LGBM (Proteomics + Metadata)')
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

# ---- Update this to match your proteomics run ----
folder = "/Users/authorname/Desktop/Projects/proteomics_LGBM(ANML+Meta)"
files = glob.glob(os.path.join(folder, "seed*_*.csv"))

# Infer class names from filenames (everything after 'seedX_')
def class_from_path(p):
    base = os.path.basename(p)
    return "_".join(base.split("_")[1:]).rsplit(".csv", 1)[0]

class_names = sorted(set(class_from_path(p) for p in files))

recall_grid = np.linspace(0.0, 1.0, 200)
plt.figure(figsize=(10, 8))

ap_summary_rows = []

for cls in class_names:
    cls_files = sorted(glob.glob(os.path.join(folder, f"seed*_{cls}.csv")))
    if not cls_files:
        continue

    prec_interp_list, aps, prevalences = [], [], []

    for path in cls_files:
        df = pd.read_csv(path)
        y_true  = df["y_true"].astype(int).values
        y_score = df["y_score"].astype(float).values

        # skip degenerate splits
        pos = y_true.sum()
        if pos == 0 or pos == len(y_true):
            continue

        precision, recall, _ = precision_recall_curve(y_true, y_score)

        # deduplicate recall for interpolation
        uniq_idx = np.unique(recall, return_index=True)[1]
        r_u, p_u = recall[uniq_idx], precision[uniq_idx]

        # interpolate precision onto common recall grid
        p_i = np.interp(recall_grid, r_u, p_u, left=p_u[0], right=p_u[-1])
        prec_interp_list.append(p_i)

        aps.append(average_precision_score(y_true, y_score))
        prevalences.append(y_true.mean())

    if not prec_interp_list:
        print(f"[WARN] No valid PR curves for class {cls}")
        continue

    mean_prec = np.mean(prec_interp_list, axis=0)
    std_prec  = np.std(prec_interp_list, axis=0)
    mean_ap   = float(np.mean(aps))
    std_ap    = float(np.std(aps))
    n_splits  = len(aps)

    label_cls = cls.replace("_", " ")
    line, = plt.plot(recall_grid, mean_prec, lw=2,
                     label=f"{label_cls} (AP = {mean_ap:.3f} ± {std_ap:.3f})")
    plt.fill_between(recall_grid,
                     np.clip(mean_prec - std_prec, 0, 1),
                     np.clip(mean_prec + std_prec, 0, 1),
                     alpha=0.15, color=line.get_color())

    # per-class chance baseline = prevalence
    prev_mean = float(np.mean(prevalences))
    prev_sd   = float(np.std(prevalences))
    plt.hlines(prev_mean, 0, 1, colors=line.get_color(), linestyles="--", alpha=0.6)
    low, high = max(0, prev_mean - prev_sd), min(1, prev_mean + prev_sd)
    plt.fill_between([0, 1], [low, low], [high, high], color=line.get_color(), alpha=0.08)

    ap_summary_rows.append({
        "Class": label_cls, "Mean_AP": mean_ap, "Std_AP": std_ap,
        "Prev_Mean": prev_mean, "Prev_SD": prev_sd, "N": n_splits
    })

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("LGBM (Proteomics + Demographics)\nPrecision–Recall (mean ± SD across seeds; dashed = prevalence)")
plt.legend(loc="upper right", fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Optional: save figure + table
plot_path = os.path.join(folder, "pr_mean_across_seeds.png")
plt.savefig(plot_path, dpi=200)
if ap_summary_rows:
    ap_df = pd.DataFrame(ap_summary_rows).sort_values("Class")
    ap_csv = os.path.join(folder, "pr_ap_summary.csv")
    ap_df.to_csv(ap_csv, index=False)
    print(f"Saved plot to: {plot_path}")
    print(f"Saved AP summary to: {ap_csv}")
    print(ap_df.to_string(index=False))